# 摘要
1. 從 reference 資料夾，載入 填值表。
2. 根據 填值表，從 Nas_data 中讀取指定圖表。
3. 根據 填值表，指定圖表之算式，將變數表中的變數，經過算式運算以後，填入 Nas_data 之中。
4. 根據 填值表，將欄位名稱全部改名。
5. 輸出 Nas_data，至此，公開資料的自動化處理結束。預期輸出正確之檔案。

In [ ]:
import pandas, numpy, os, traceback

In [ ]:
# 載入填質表
reference_excel_path = os.path.join(os.path.join(os.path.dirname(os.getcwd()), "reference"), "填值表.xlsx")
reference_excel = pandas.read_excel(reference_excel_path)
reference_excel.head(3)

In [ ]:
Nas_data_path = os.path.join(os.path.join(os.path.dirname(os.getcwd()), "reference"), "Step1_data.xlsx")
Nas_data = pandas.read_excel(Nas_data_path)
Nas_data[Nas_data["學校代碼"] == "0001"]

In [ ]:
# 載入資料庫
# 載入資料庫
data_base_path = os.path.join(os.path.join(os.path.dirname(os.getcwd()), "reference"), "Step3_data_base.xlsx")
data_base = pandas.read_excel(data_base_path)
data_base = data_base[data_base["學年度"].notnull()]
data_base["學年度"] = data_base["學年度"].astype(int) # 將學年度轉為字串型別。以配合後續的填值。
data_base["學年度"] = data_base["學年度"].astype(str)
data_base[data_base["學校代碼"] == "0001"]


# 填值

## 可先串接之資料庫

In [ ]:
# 創建一個新的 Excel Writer 對象
writer_path = os.path.join(os.path.join(os.path.dirname(os.getcwd()), "reference"), "Step4_check_data.xlsx")
with pandas.ExcelWriter(writer_path, engine="openpyxl") as writer :

    for picture_name in list(reference_excel["使用圖型"]) : 
        # 處理可先串接之圖型
        if reference_excel[reference_excel["使用圖型"] == picture_name]["能否先串接"].values[0] == "Yes" : 
            try : 
                Nas_picture_data = pandas.read_excel(Nas_data_path, sheet_name= picture_name)
                
                # 為每一張圖表，新增一個新的學年度欄位。如，原本只有 110 年，則多一欄 111年。
                Nas_picture_data[str(int(Nas_picture_data.columns[-1]) + 1)] = 0

                # 載入該圖表對應的公式。備註，這個是字串型別的公式，要轉譯才能使用。
                formula = reference_excel[reference_excel["使用圖型"] == picture_name]["公式(pandas)"].values[0]
                
                # 將公式字串做轉換，data["???"]，轉為，sch_open_data["???"]
                formula = formula.replace("data", "sch_year_data")
                formula = formula.replace("]", "].values[0]")

                # text : 
                # print(picture_name)
                # print(formula)

                try : 
                    # 根據公式，將公開資料庫的數據，填入Nas_data之中。
                    for sch_index in list(dict.fromkeys(Nas_picture_data["學校代碼"])) : 
                        sch_open_data = data_base[data_base["學校代碼"] == sch_index]


                        for sch_year in list(dict.fromkeys(sch_open_data["學年度"])) : 
                            sch_year_data = sch_open_data[sch_open_data["學年度"] == sch_year]
                            # 填入值
                            input_number = round(eval(formula), 9) # 這一行可能會報錯，預期錯誤為，IndexError: index 0 is out of bounds for axis 0 with size 0。但應該不影響結果。

                            try : # 將 sch_open_data 中，存在的學年度，盡可能填入 Nas_picture_data 之中。

                                if str(int(sch_year)) in Nas_picture_data.columns : # 僅寫入 Nas_picture_data 中，有的學年度資料。
                                    if reference_excel[reference_excel["使用圖型"] == picture_name]["上限為100"].values[0] == "Yes" : # 選出使用百分比單位的圖表。# 註冊率超過100%者，以100%計算。
                                        if input_number > 100 :
                                            input_number = 100
                                            Nas_picture_data.loc[Nas_picture_data["學校代碼"] == sch_index, str(int(sch_year))] = input_number # eval() 轉譯用的函式。
                                        else : 
                                            Nas_picture_data.loc[Nas_picture_data["學校代碼"] == sch_index, str(int(sch_year))] = input_number
                                    else : 
                                        Nas_picture_data.loc[Nas_picture_data["學校代碼"] == sch_index, str(int(sch_year))] = input_number
                                
                                
                            except Exception as e : 
                                error_message = traceback.format_exc()
                                with open(os.path.join(os.path.dirname(os.getcwd()), "error", "可以先串接資料庫(填值_Error)"), "w", encoding='utf-8') as f:
                                    f.write(error_message)
            
                except Exception as e : 
                    error_message = traceback.format_exc()
                    with open(os.path.join(os.path.dirname(os.getcwd()), "error", "公式計算錯誤"), "w", encoding='utf-8') as f:
                        f.write(error_message)


                # 更改欄位名稱，加上填值表的變數名稱。
                column_name = reference_excel[reference_excel["使用圖型"] == picture_name]["變數名稱"].values[0]
                # 每一個分頁的第五行開始，皆是數值變數。
                sch_year_column_list = Nas_picture_data.columns[4:]

                # 欄位改名之結果 list。
                rename_list = [column_name + "_" + str(sch_year_column) for sch_year_column in sch_year_column_list]

                rename_dict = dict(zip(sch_year_column_list, rename_list))
                # print(rename_dict)

                Nas_picture_data = Nas_picture_data.rename(columns = rename_dict)

                # 遺漏值以 0 取代。
                Nas_picture_data = Nas_picture_data.fillna(0) # 這裡推薦使用 0 ，方便後面的除錯。

                # 將無窮大數值，以NA取代。
                # Nas_picture_data.replace(numpy.inf, "NA")

                # 儲存檔案
                Nas_picture_data.to_excel(writer, sheet_name = picture_name, index = False)
            except Exception as e : 
                error_message = traceback.format_exc()
                with open(os.path.join(os.path.dirname(os.getcwd()), "error", "圖型寫入錯誤"), "w", encoding='utf-8') as f:
                    f.write(error_message)
            



## 處理不可先串接之資料庫

# 資料除錯

## 可先串接之資料除錯

In [ ]:
# 載入過去的 Nas_data。即 - Nas_data_check。透過對比，Nas_data_check，與 Nas_data之間的差異，找出與往年不同的資料。
Nas_data_path = os.path.join(os.path.join(os.path.dirname(os.getcwd()), "reference"), "Step4_check_data.xlsx")
Nas_check_data_path = os.path.join(os.path.join(os.path.dirname(os.getcwd()), "reference"), "Step2_data_check.xlsx")

error_list = [] # 這個用來儲存錯誤資訊。

for picture_name in list(reference_excel["使用圖型"]) : # 可以知道的是，目前有資料的部分，都通過了 try 的內容。
	
	# 處理能先串接之圖型。
	if reference_excel[reference_excel["使用圖型"] == picture_name]["能否先串接"] == "Yes" : 
		try : 
		# print(picture_name)
			Nas_picture_data = pandas.read_excel(Nas_data_path, sheet_name = picture_name)
			# 排除最新的資料
			Nas_picture_data = Nas_picture_data.iloc[:, :-1]

			Nas_picture_check_data = pandas.read_excel(Nas_check_data_path, sheet_name = picture_name)

			# 比較除了最新資料以外。每校，每年之數值，是否相同。不同者，存到 error_list。需要資訊，檔案名稱、學校代碼、欄位名稱。
			for sch_index in list(Nas_picture_data["學校代碼"]) : # 抓出兩檔案中，相同學校之資料。
				sch_index_data = Nas_picture_data[Nas_picture_data["學校代碼"] == sch_index]
				sch_index_check_data = Nas_picture_check_data[Nas_picture_check_data["學校代碼"] == sch_index]

				for column_name in Nas_picture_data.columns[4:] : 
					if abs(sch_index_data[column_name].values[0] - sch_index_check_data[column_name].values[0]) > 10**-5 : 
						error_list.append({"檔案名稱" : picture_name, "學校代碼" : sch_index, "欄位名稱" : column_name, 
											"當前值" : sch_index_data[column_name].values[0],
											"過去值" : sch_index_check_data[column_name].values[0], 
											"差異量" : sch_index_data[column_name].values[0] - sch_index_check_data[column_name].values[0]})
		except : 
			print("還沒弄得圖型")
			print(picture_name) # 還沒弄的，儲存起來。# 那些可以跑，那些不能。

		

		


## 不可先串接之資料除錯

# 不同數值輸出

In [ ]:
error_data = pandas.DataFrame(error_list)
error_data_path = os.path.join(os.path.join(os.path.dirname(os.getcwd()), "error"), "數值不同.xlsx")
error_data.to_excel(error_data_path, index = False)

In [ ]:
error_data # 為空意味著，當前所蒐集的所有數值均與過去蒐集的均相同。如果為空，則可以進行檔案輸出。

# 底下的零，可能原本是 numpy.nan，只是需要先轉為 0，才能夠與過去數值比較。


